In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load the data
train_images = np.load('train_images.npy')
train_labels = np.load('train_labels.npy')

In [ ]:
# Reshape the images to be 3-dimensional (channels_last)
train_images = train_images.reshape(-1, 224, 224, 3)

In [ ]:
# Normalize the pixel values
train_images = train_images / 255.0

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow(
    train_images,
    train_labels,
    batch_size=32,
    subset='training'
)

In [ ]:
# Model definition
base_model = keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)

x = base_model.output
x = keras.layers.Dense(512, activation='relu')(x)
predictions = keras.layers.Dense(1)(x)

model = keras.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='mse')

In [ ]:
# Model training
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=train_generator,
    validation_steps=len(train_generator) // 10,
    steps_per_epoch=len(train_generator) // 32
)

In [ ]:
# Evaluation
losses = history.history['loss']
val_losses = history.history['val_loss']

epochs = range(len(losses))

plt.figure(figsize=(10, 6))
plt.plot(epochs, losses, 'b', label='Training loss')
plt.plot(epochs, val_losses, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()`